In [2]:
import json

DATA_FOLDER = "data"

urls = dict()
with open(f'{DATA_FOLDER}/followers_rehydrated.jsonl') as i:
    for l in i:
        j = json.loads(l)
        urls[j['id']] = j['profile_image_url']

with open(f'{DATA_FOLDER}/follower_pic_urls.csv', 'w+') as o:
    o.write('\n'.join(f"{k},{v}" for k, v in urls.items()))

In [3]:
len(urls), len(set(urls.values()))

(5456127, 4139655)

In [11]:
import requests
import shutil
from urllib.parse import urlparse
import os

errors = []

for k, v in urls.items():
    try:
        path = urlparse(v).path
        ext = os.path.splitext(path)[1]
        r = requests.get(v, stream=True)
    except:
        errors.append(k)
    if r.status_code == 200: #TODO: modify so that it gets the full size img
        out_path = f"{DATA_FOLDER}/img/{k[:2]}/{k[2:4]}/{k}{ext}"
        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        with open(out_path, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f)
    else:
        errors.append(k)

MissingSchema: Invalid URL '': No scheme supplied. Perhaps you meant https://?

In [12]:
list(urls.values())[:3]

['https://pbs.twimg.com/profile_images/852969926163808256/SaSysP9k_normal.jpg',
 'https://pbs.twimg.com/profile_images/1454018403274948610/9QgH-Lj3_normal.jpg',
 'https://pbs.twimg.com/profile_images/640041222082465792/4g-y6Ztk_normal.jpg']